# Прогнозируем задержки самолётов


В этом домашнем задании вы будете работать с задачей про задержки самолётов.  На основании доступных данных о рейсе вам нужно определить, будет ли он задержан на 15 минут.

## Импорт библиотек, установка констант

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import roc_auc_score

In [ ]:
RANDOM_STATE = 111
DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

## Загрузка и обзор данных

In [ ]:
data = pd.read_csv(DATASET_PATH)

In [ ]:
data.sample(5)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
78644,c-10,c-3,c-1,1605,UA,DEN,MSP,680,Y
47992,c-4,c-3,c-1,1506,US,PHX,ABQ,328,N
37187,c-6,c-6,c-1,1240,WN,OAK,LAS,407,N
69587,c-2,c-28,c-2,651,XE,BWI,EWR,169,N
90674,c-7,c-18,c-2,1130,AA,BDL,ORD,783,N


In [ ]:
chisl_data = data.select_dtypes(['int64', 'float64'])
chisl_data

,DepTime,Distance
0,1934,732
1,1548,834
2,1422,416
3,1015,872
4,1828,423
...,...,...
99995,1618,199
99996,804,884
99997,1901,1076
99998,1515,140


## Вопрос для Quiz

Сколько числовых столбцов в данных?

## Обучение моделей

Разбейте данные
* на матрицу объект признак `X`, содержащую только числовые столбцы `DepTime`, `Distance`
* вектор с целевой переменной `y`, равный столбцу `dep_delayed_15min`, переведенном в 0 и 1 по правилу: 'Y' - 1, 'N' - 0.

In [ ]:
X = data[['DepTime', 'Distance']]
y = data['dep_delayed_15min'].map({'Y' : 1, 'N' : 0})
y

0        0
1        0
2        0
3        0
4        1
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Name: dep_delayed_15min, Length: 100000, dtype: int64

Обучите по кросс-валидации с тремя фолдами и метрикой *ROC-AUC* (`scoring='roc_auc'`) три модели с гиперпараметрами по умолчанию:
* решающее дерево
* случайный лес
* градиентный бустинг

Выведите на экран среднее качество по кросс-валидации каждой из моделей.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
dec_tree = DecisionTreeClassifier(random_state=RANDOM_STATE)
print(cross_val_score(dec_tree, X, y, cv=3, scoring='roc_auc').mean())
ran_for = RandomForestClassifier(random_state=RANDOM_STATE)
print(cross_val_score(ran_for, X, y, cv=3, scoring='roc_auc').mean())
grad = GradientBoostingClassifier(random_state=RANDOM_STATE)
print(cross_val_score(grad, X, y, cv=3, scoring='roc_auc').mean())

0.5792982351791407
0.6758129274941099
0.6992516986788738


## Вопрос для Quiz

какой алгоритм показал наилучшее качество?

Выведите *bias* и *variance* каждой из моделей при помощи `bias_variance_decomp` из библиотеки `mlxtend`. Функция на вход ожидает получения тренировочных и тестовых данных, поэтому разобъем все данные на train и test.

Для ускорения вычислений
* возьмите только первые 1000 строк из тренировочных данных (`Xtrain[:1000], ytrain[:1000]`)
* поставьте `num_rounds = 50` в функции `bias_variance_decomp`

И не забудьте перевести `pd.dataframe` в `np.array` - так как библиотека умеет работать только с этим типом данных (`Xtrain.values` и так для всех объектов).

In [ ]:
!pip install mlxtend --upgrade

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

X_train_new = Xtrain[:1000].values
X_test_new = Xtest.values
y_train_new = ytrain[:1000].values
y_test_new = ytest.values
avg_mse, avg_bias, avg_var = bias_variance_decomp(dec_tree, X_train_new, y_train_new,
                                                  X_test_new, y_test_new, num_rounds=50,
                                                  random_seed=np.random.seed(RANDOM_STATE))
print(avg_mse, avg_bias, avg_var)
avg_mse, avg_bias, avg_var = bias_variance_decomp(ran_for, X_train_new, y_train_new,
                                                  X_test_new, y_test_new, num_rounds=50,
                                                  random_seed=np.random.seed(RANDOM_STATE))
print(avg_mse, avg_bias, avg_var)
avg_mse, avg_bias, avg_var = bias_variance_decomp(grad, X_train_new, y_train_new,
                                                  X_test_new, y_test_new, num_rounds=50,
                                                  random_seed=np.random.seed(RANDOM_STATE))
print(avg_mse, avg_bias, avg_var)

0.2972432 0.24772 0.17077120000000004
0.24600240000000004 0.22756 0.08958799999999999
0.21658480000000002 0.19736 0.0630776


## Подбор гиперпараметров

Напомним, как мы действуем при подборе гиперпараметров:
* разбиваем данные на *train* и *test*
* на train по кросс-валидации при помощи `GridSearchCV` ищем наилучшие гиперпараметры модели
* оцениваем качество модели с наилучшими гиперпараметрами на *test*

Будем использовать 3 фолда при кросс-валидации, а метрику - *ROC-AUC* (`scoring='roc_auc'`).


Далее везде можно обучать модель на первых 10000 строк из обучающей выборки (так как всего два признака).

### 1. Подберите гиперпараметры для решающего дерева:
* `max_depth` (обязательно)
* `max_features` (обязательно)
* `criterion` (по желанию)
* `min_samples_split` (по желанию)
* `min_samples_leaf` (по желанию)

Диапазоны поиска гиперпараметров выбирайте самостоятельно.

Добейтесь качества *ROC-AUC* $>0.65$ на кросс-валидации.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
params = {'criterion' : ('gini', 'entropy'), 'max_features' : range(1, 3), 'max_depth'  :range(1, 10, 2) }
gs = GridSearchCV(dec_tree, params, cv=3, scoring='roc_auc' )
gs.fit(Xtrain[:10000], ytrain[:10000])
gs.best_score_, gs.best_estimator_
tree = gs.best_estimator_
y_pred = tree.predict_proba(Xtest)[:, 1]
print(roc_auc_score(ytest, y_pred))

0.6849701531145349


Теперь обучите на **всех** тренировочных данных решающее дерево с лучшими  
найденными гиперпараметрами и выведите *ROC-AUC* на тесте.

In [ ]:
tree.fit(Xtrain, ytrain)
tree_pred = tree.predict_proba(Xtest)[:, 1]
print(roc_auc_score(ytest, tree_pred))

0.6922765202446044


### 2. Подберите гиперпараметры для случайного леса:
* `n_estimators` (обязательно)
* `max_depth` (обязательно)
* `max_features` (по желанию)
* `criterion` (по желанию)
* `min_samples_split` (по желанию)
* `min_samples_leaf` (по желанию)

Диапазоны поиска гиперпараметров выбирайте самостоятельно.

Добейтесь качества *ROC-AUC* $>0.65$ на кросс-валидации.

In [ ]:
params = {'n_estimators': np.arange(10, 200, 20),
          'max_depth': np.arange(2, 20, 3),
          'criterion' : ('gini', 'entropy')}
ggs = GridSearchCV(ran_for, params, cv=3, scoring='roc_auc')
ggs.fit(Xtrain[:10000], ytrain[:10000])
ggs.best_estimator_, ggs.best_score_

(RandomForestClassifier(max_depth=5, n_estimators=190, random_state=111),
 0.6989616688802819)

Теперь обучите на **всех** тренировочных данных случайный лес с лучшими  
найденными гиперпараметрами и выведите *ROC-AUC* на тесте.

In [ ]:
new_forest = ggs.best_estimator_
new_forest.fit(Xtrain, ytrain)
forest_pred = new_forest.predict_proba(Xtest)[:, 1]
print(roc_auc_score(ytest, forest_pred))

0.7033538250996584


### 3. Подберите гиперпараметры для градиентного бустинга:
* `n_estimators` (обязательно)
* `max_depth` (обязательно)
* `max_features` (по желанию)
* `criterion` (по желанию)
* `min_samples_split` (по желанию)
* `min_samples_leaf` (по желанию)

Диапазоны поиска гиперпараметров выбирайте самостоятельно.

Добейтесь качества *ROC-AUC* $>0.65$ на кросс-валидации.

In [ ]:
params = {'n_estimators': np.arange(10, 200, 20),
          'max_depth': np.arange(2, 20, 3)}
gg = GridSearchCV(grad, params, cv=3, scoring='roc_auc')
gg.fit(Xtrain[:10000], ytrain[:10000])
gg.best_estimator_, gg.best_score_

(GradientBoostingClassifier(max_depth=2, n_estimators=30, random_state=111),
 0.6994624405848318)

Теперь обучите на **всех** тренировочных данных градиентный бустинг с лучшими  
найденными гиперпараметрами и выведите *ROC-AUC* на тесте.

In [ ]:
gra = gg.best_estimator_
gra.fit(Xtrain, ytrain)
gra_pred = gra.predict_proba(Xtest)[:, 1]
print(roc_auc_score(ytest, gra_pred))

0.6997204913343631
